In [ ]:
# 初始化
import django_setup

In [ ]:
from apps.projects.models import Project, Task, TaskType
from pprint import pprint
project = Project.objects.get(project_name='测试项目1')
task = Task.objects.get(stage__project=project, type=TaskType.OUTLINE_ANALYSIS_TASK)
stage = task.stage
# --- 如果project.tender_file_extraction 为空，则需要从关联的docx_extraction_task中获取
# extraction_task = Task.objects.get(stage__project=project, type=TaskType.DOCX_EXTRACTION_TASK)
# print(extraction_task.docx_tiptap)
# print(type(extraction_task.docx_tiptap))
# pprint(extraction_task.docx_tiptap)
# project = Project.objects.get(project_name='测试项目1')
# project.tender_file_extraction = extraction_task.docx_tiptap
# project.save()
# project = Project.objects.get(project_name='测试项目1')
pprint(project.tender_file_extraction)

#### 1. 分析前期准备
<font color="orange">
为大模型准备分析所需要的：context, instruction, supplement, output_format, prompt_template, index_path_map <br>
直接调用services/tasks_preparation/outline_analysis.prepare_for_task <br>

整合思考： prepare_for_task 应该放在执行链路的哪个环节？如何触发？ <br>
1）任务初建，在signal和初始化一起执行，但这时context关联素材还没ready? （可考虑用“提示说明”来解决）-> 创建初始化时触发。<br>
2）然后, 在关联任务完成时，执行prepare_for_task。  这步无论如何需要执行。 （关联任务完成时，触发）<br>
3）只有这样才能执行之后的task任务<br>
</font>

In [ ]:
from apps.projects.services.tasks_preparation.outline_analysis import OutlineAnalysis
task_ready=OutlineAnalysis(project.id).prepare_for_task()

In [6]:
# 需要重新查询，才能看到prepare_for_task的执行结果
task = Task.objects.get(stage__project=project, type=TaskType.OUTLINE_ANALYSIS_TASK)

In [ ]:
print(type(task.llm_config), task.llm_config)

In [ ]:
print(task.llm_config['llm_model_name'])
print(task.llm_config['temperature'])
print(task.llm_config['top_p'])
print(task.llm_config['streaming'])
print(task.llm_config['base_url'])
print(task.llm_config['max_workers'])
print(task.llm_config['timeout'])
print(task.llm_config['retry_times'])

In [ ]:
print(type(task.context), task.context)

In [ ]:
print(type(task.instruction), task.instruction)
print(type(task.supplement), task.supplement)
print(type(task.output_format), task.output_format)
print(type(task.prompt_template), task.prompt_template)
print(type(task.index_path_map), task.index_path_map)



In [ ]:
print(type(task.context_tokens), task.context_tokens)
print(type(task.instruction_tokens), task.instruction_tokens)
print(type(task.supplement_tokens), task.supplement_tokens)
print(type(task.output_format_tokens), task.output_format_tokens)
print(type(task.prompt_template_tokens), task.prompt_template_tokens)
print(type(task.in_tokens), task.in_tokens)

In [ ]:
# 模拟prompt的生成， 调用services/tasks_preparation/online_analysis.simulate_prompt
from apps.projects.services.tasks_preparation.outline_analysis import OutlineAnalysis
from pprint import pprint
prompt = OutlineAnalysis(project.id).simulate_prompt()
print(prompt)
pprint(prompt)


#### 2. 大模型分析
<font color="orange">
大模型分析的素材： context, instruction, supplement, output_format, prompt_template, index_path_map 已经ready <br>
直接调用services/LLM_server/LLM_task_container.py 的LLM_task_container.run_task <br>
</font>

In [ ]:
# 更新task状态为PROCESSING， 为调用任务分析做状态准备。 
from apps.projects.models import Project, Task, TaskType, TaskStatus
project = Project.objects.get(project_name='测试项目1')
task = Task.objects.get(stage__project=project, type=TaskType.OUTLINE_ANALYSIS_TASK)
print('当前状态：',task.status)
task.status = TaskStatus.PROCESSING
task.save()
task = Task.objects.get(stage__project=project, type=TaskType.OUTLINE_ANALYSIS_TASK)
print('更新后状态：',task.status)


In [ ]:
# 模拟task_streaming.py 视图文件 中 start_stream 方法 启动大模型分析任务 
from apps.projects.tasks import process_task_analysis_streaming_v2
print(f"开始调用任务：{project.id}, {stage.stage_type}, {task.type}")
celery_task = process_task_analysis_streaming_v2.delay(project.id, stage.stage_type, task.type)
print(f"任务调用完成：{celery_task.id}")

In [ ]:
# 模拟task_streaming.py 视图文件 中 get_stream_result 方法 获取大模型分析任务结果
from apps.projects.utils.redis_manager import RedisManager
from pprint import pprint
from IPython.display import display, HTML, Markdown


redis_manager = RedisManager()
chunks = redis_manager.get_stream_chunks(celery_task.id)
# 过滤掉特殊标记块
content_chunks = [
    chunk for chunk in chunks 
    if chunk.get('content') != 'DONE' and not chunk.get('content', '').startswith('ERROR:')
]
# 合并内容
full_content = ''.join([chunk.get('content', '') for chunk in content_chunks])
pprint(full_content)
display(Markdown(full_content))


In [ ]:
# 检查: 在任务执行完成后，task.final_result 是否被更新 （分析结果被转为tiptap_content 存储进task.final_result 的代码在tasks.py中）。 
from apps.projects.models import Project, Task, TaskType, TaskStatus
project = Project.objects.get(project_name='测试项目1')
task = Task.objects.get(stage__project=project, type=TaskType.OUTLINE_ANALYSIS_TASK)
print('final_result',task.final_result)
pprint(task.final_result)